In [54]:
%logstop
%logstart -rtq ~/.logs/ml.py append
import seaborn as sns
sns.set()

In [3]:
from static_grader import grader

In [36]:
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.dpi'] = 144

# ML Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to create effective machine learning models for making predictions. We will be working with nursing home inspection data from the United States, predicting which providers may be fined and for how much.

## Scoring

In this miniproject you will often submit your model's `predict` or `predict_proba` method to the grader. The grader will assess the performance of your model using a scoring metric, comparing it against the score of a reference model. We will use the [average precision score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html). If your model performs better than the reference solution, then you can score higher than 1.0.

**Note:** If you use an estimator that relies on random draws (like a `RandomForestClassifier`) you should set the `random_state=` to an integer so that your results are reproducible. 

## Downloading the data

We can download the data set from Amazon S3:

In [4]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-train.csv -nc -P ./ml-data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-metadata.csv -nc -P ./ml-data

mkdir: cannot create directory ‘data’: File exists
File ‘./ml-data/providers-train.csv’ already there; not retrieving.

File ‘./ml-data/providers-metadata.csv’ already there; not retrieving.



We'll load the data into a Pandas DataFrame. Several columns will become target labels in future questions. Let's pop those columns out from the data, and drop related columns that are neither targets nor reasonable features (i.e. we don't wouldn't know how many times a facility denied payment before knowing whether it was fined).

The data has many columns. We have also provided a data dictionary.

In [5]:
import numpy as np
import pandas as pd

In [6]:
metadata = pd.read_csv('./ml-data/providers-metadata.csv')
metadata.head()

,Variable,Label,Description,Format
0,PROVNUM,Federal Provider Number,Federal Provider Number,6 alphanumeric characters
1,PROVNAME,Provider Name,Provider Name,text
2,ADDRESS,Provider Address,Provider Address,text
3,CITY,Provider City,Provider City,text
4,STATE,Provider State,Provider State,2-character postal abbreviation


In [7]:
data = pd.read_csv('./ml-data/providers-train.csv', encoding='latin1')

fine_counts = data.pop('FINE_CNT')
fine_totals = data.pop('FINE_TOT')
cycle_2_score = data.pop('CYCLE_2_TOTAL_SCORE')

In [8]:
metadata

,Variable,Label,Description,Format
0,PROVNUM,Federal Provider Number,Federal Provider Number,6 alphanumeric characters
1,PROVNAME,Provider Name,Provider Name,text
2,ADDRESS,Provider Address,Provider Address,text
3,CITY,Provider City,Provider City,text
4,STATE,Provider State,Provider State,2-character postal abbreviation
...,...,...,...,...
74,FINE_TOT,Total Amount of Fines in Dollars,Total Amount of Fines in Dollars,integer
75,PAYDEN_CNT,Number of Payment Denials,Number of Payment Denials,integer
76,TOT_PENLTY_CNT,Total Number of Penalties,Total Number of Penalties,integer
77,LOCATION,Location,Location of facility,only on displayed version of file; renders as ...


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13892 entries, 0 to 13891
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PROVNUM                13892 non-null  object 
 1   PROVNAME               13892 non-null  object 
 2   ADDRESS                13892 non-null  object 
 3   CITY                   13892 non-null  object 
 4   STATE                  13892 non-null  object 
 5   ZIP                    13892 non-null  int64  
 6   PHONE                  13892 non-null  int64  
 7   COUNTY_SSA             13892 non-null  int64  
 8   COUNTY_NAME            13892 non-null  object 
 9   BEDCERT                13892 non-null  int64  
 10  RESTOT                 13483 non-null  float64
 11  INHOSP                 13892 non-null  bool   
 12  CCRC_FACIL             13892 non-null  bool   
 13  SFF                    13892 non-null  bool   
 14  CHOW_LAST_12MOS        13892 non-null  bool   
 15  SP

## Question 1: state_model

A federal agency, Centers for Medicare and Medicaid Services (CMS), imposes regulations on nursing homes. However, nursing homes are inspected by state agencies for compliance with regulations, and fines for violations can vary widely between states.

Let's develop a very simple initial model to predict the amount of fines a nursing home might expect to pay based on its location. Fill in the class definition of the custom estimator, `StateMeanEstimator`, below.

**Note:** When the grader checks your answer, it passes a list of dictionaries to the `predict` method of your estimator, not a DataFrame. This means that your model must work with both data types. You can handle this by adding a test (and optional conversion) in the `predict` method of your custom class, similar to the `ColumnSelectTransformer` given below in Question 2.  

In [10]:
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin

class GroupMeanEstimator(BaseEstimator, RegressorMixin):
    def __init__(self, grouper):
        self.grouper = grouper
        self.group_averages = {}

    def fit(self, X, y):    
        # Use self.group_averages to store the average penalty by group
        Xt = X.copy()
        Xt['fines'] = y
        self.group_averages_ = dict(Xt.groupby(self.grouper)['fines'].mean())
        self.y_mean_ = y.mean()
        return self

    def predict(self, X): 
        # make sure test data is a pandas dataframe
        Xt = pd.DataFrame(X)
        # allow for unknown states: impute with mean val
        for state in Xt[self.grouper]:
            if state not in self.group_averages_.keys():
                self.group_averages_[state] = self.y_mean_
        # Return a list of predicted penalties based on group of samples in X
        return [self.group_averages_[x] for x in Xt[self.grouper]]

After filling in class definition, we can create an instance of the estimator and fit it to the data.

In [11]:
from sklearn.pipeline import Pipeline

state_model = Pipeline([
    ('sme', GroupMeanEstimator(grouper='STATE'))
    ])
state_model.fit(data, fine_totals)

Pipeline(steps=[('sme', GroupMeanEstimator(grouper='STATE'))])

Next we should test that our predict method works.

In [12]:
state_model.predict(data.sample(5))

[5626.954545454545,
 18425.87278106509,
 12365.983606557376,
 8214.822977725675,
 18147.119744058502]

However, what if we have data from a nursing home in a state (or territory) of the US which is not in the training data?

In [13]:
state_model.predict(pd.DataFrame([{'STATE': 'AS'}]))

[14969.857687877915]

Make sure your model can handle this possibility before submitting your model's predict method to the grader.

In [14]:
grader.score.ml__state_model(state_model.predict)

Your score: 1.000


## Question 2: simple_features_model

Nursing homes vary greatly in their business characteristics. Some are owned by the government or non-profits while others are run for profit. Some house a few dozen residents while others house hundreds. Some are located within hospitals and may work with more vulnerable populations. We will try to predict which facilities are fined based on their business characteristics.

We'll begin with columns in our DataFrame containing numeric and boolean features. Some of the rows contain null values; estimators cannot handle null values so these must be imputed or dropped. We will create a `Pipeline` containing transformers that process these features, followed by an estimator.

**Note:** As mentioned above in Question 1, when the grader checks your answer, it passes a list of dictionaries to the `predict` or `predict_proba` method of your estimator, not a DataFrame. This means that your model must work with both data types. For this reason, we've provided a custom `ColumnSelectTransformer` for you to use instead `scikit-learn`'s own `ColumnTransformer`.

In [15]:
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin
from sklearn.pipeline import Pipeline

simple_cols = ['BEDCERT', 'RESTOT', 'INHOSP',
               'CCRC_FACIL', 'SFF', 'CHOW_LAST_12MOS',
               'SPRINKLER_STATUS', 'EXP_TOTAL', 'ADJ_TOTAL']

class ColumnSelectTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        Xt = X[self.columns].copy() # copy() to prevent SettingWithCopyWarning
        
        return Xt

class MeanFillna(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # impute floats into the columns with nans
        # use global mean values
        X[self.columns].fillna(value=X[self.columns].mean(), inplace=True)
        
        return X
        
simple_features = Pipeline([
    ('cst', ColumnSelectTransformer(simple_cols)),
    ('fillna1', MeanFillna('RESTOT')),
    ('fillna2', MeanFillna('EXP_TOTAL')),
    ('fillna3', MeanFillna('ADJ_TOTAL')),
])

**Note:** The assertion below assumes the output of `noncategorical_features.fit_transform` is a `ndarray`, not a `DataFrame`.)

In [16]:

assert data['RESTOT'].isnull().sum() > 0
assert not np.isnan(simple_features.fit_transform(data).values.any())

Now combine the `simple_features` pipeline with an estimator in a new pipeline. Fit `simple_features_model` to the data and submit `simple_features_model.predict_proba` to the grader. You may wish to use cross-validation to tune the hyperparameters of your model.

In [17]:
from sklearn.linear_model import LogisticRegression

In [18]:
from sklearn import model_selection, utils

In [19]:
#create class weights to improve perfomance
class_weights = utils.class_weight.compute_class_weight(
    'balanced',np.unique(fine_counts > 0),fine_counts > 0)
classes = np.unique(fine_counts > 0)
class_weights = dict(zip(classes,class_weights))

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass classes=[False  True], y=0         True
1        False
2        False
3        False
4        False
         ...  
13887    False
13888    False
13889    False
13890    False
13891     True
Name: FINE_CNT, Length: 13892, dtype: bool as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [20]:
simple_features_model = Pipeline([
    ('simple', simple_features),
    # add your estimator here
    ('clf', LogisticRegression(random_state=42, max_iter=500, solver='lbfgs', class_weight=class_weights))
])

In [21]:
simple_features_model.fit(data, fine_counts > 0)

Pipeline(steps=[('simple',
                 Pipeline(steps=[('cst',
                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                   'RESTOT',
                                                                   'INHOSP',
                                                                   'CCRC_FACIL',
                                                                   'SFF',
                                                                   'CHOW_LAST_12MOS',
                                                                   'SPRINKLER_STATUS',
                                                                   'EXP_TOTAL',
                                                                   'ADJ_TOTAL'])),
                                 ('fillna1', MeanFillna(columns='RESTOT')),
                                 ('fillna2', MeanFillna(columns='EXP_TOTAL')),
                                 ('fillna3',
         

In [22]:
# grader.score.ml__simple_features(positive_probability(simple_features_model))

def positive_probability(model):
    def predict_proba(X):
        return model.predict_proba(X)[:, 1]
    return predict_proba

grader.score.ml__simple_features(positive_probability(simple_features_model))

Your score: 1.012


## Question 3: categorical_features

The `'OWNERSHIP'` and `'CERTIFICATION'` columns contain categorical data. We will have to encode the categorical data into numerical features before we pass them to an estimator. Construct one or more pipelines for this purpose. Transformers such as [LabelEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder) and [OneHotEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder) may be useful, but you may also want to define your own transformers.

If you used more than one `Pipeline`, combine them with a `FeatureUnion`. As in Question 2, we will combine this with an estimator, fit it, and submit the `predict_proba` method to the grader.

In [23]:
from sklearn.pipeline import FeatureUnion

from sklearn.preprocessing import OneHotEncoder

owner_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['OWNERSHIP'])),
    ('ohe', OneHotEncoder (categories = 'auto', sparse = False))
])

cert_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['CERTIFICATION'])),
    ('ohe', OneHotEncoder (categories = 'auto', sparse = False))
])

categorical_features = FeatureUnion([
    ('owner', owner_onehot),
    ('cert', cert_onehot)
])

In [24]:
assert categorical_features.fit_transform(data).shape[0] == data.shape[0]
assert categorical_features.fit_transform(data).dtype == np.float64
assert not np.isnan(categorical_features.fit_transform(data)).any()

As in the previous question, create a model using the `categorical_features`, fit it to the data, and submit its `predict_proba` method to the grader.

In [25]:
#create class weights to improve perfomance
from sklearn import model_selection, utils

class_weights = utils.class_weight.compute_class_weight(
    'balanced',np.unique(fine_counts > 0),fine_counts > 0)
classes = np.unique(fine_counts > 0)
class_weights = dict(zip(classes,class_weights))

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass classes=[False  True], y=0         True
1        False
2        False
3        False
4        False
         ...  
13887    False
13888    False
13889    False
13890    False
13891     True
Name: FINE_CNT, Length: 13892, dtype: bool as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [26]:
from sklearn.ensemble import RandomForestClassifier

In [33]:
categorical_features_model = Pipeline([
    ('categorical', categorical_features),
    # add your estimator here
    ('classifier', RandomForestClassifier())
])

In [34]:
categorical_features_model.fit(data, fine_counts > 0)

Pipeline(steps=[('categorical',
                 FeatureUnion(transformer_list=[('owner',
                                                 Pipeline(steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['OWNERSHIP'])),
                                                                 ('ohe',
                                                                  OneHotEncoder(sparse=False))])),
                                                ('cert',
                                                 Pipeline(steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['CERTIFICATION'])),
                                                                 ('ohe',
                                                                  OneHotEncoder(sparse=False))]))])),
                ('classifier', RandomForestClassifier())])

In [35]:
grader.score.ml__categorical_features(positive_probability(categorical_features_model))

Your score: 0.934


## Question 4: business_model

Finally, we'll combine `simple_features` and `categorical_features` in a `FeatureUnion`, followed by an estimator in a `Pipeline`. You may want to optimize the hyperparameters of your estimator using cross-validation or try engineering new features (e.g. see [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)). When you've assembled and trained your model, pass the `predict_proba` method to the grader.

In [55]:
from sklearn.preprocessing import PolynomialFeatures
from  sklearn.linear_model import LogisticRegression

class Encoder(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        X[self.columns].replace(to_replace=[True, False], value=[1,0], inplace=True)
        
        return X

business_feature = FeatureUnion([
    ('simple', simple_features),
    ('categorical', categorical_features)
])

In [59]:

# business_feature = FeatureUnion([
#     ('simple', simple_features),
#     ('categorical', categorical_features)
# ])
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore',category=DataConversionWarning)

In [63]:
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostClassifier

In [64]:
from sklearn.tree import DecisionTreeClassifier

In [66]:
business_model = Pipeline([
    ('features', business_features),
    ('encoder1', Encoder(2)), #INHOSP
    ('encoder2', Encoder(3)), #CCRC_FACIL
    ('encoder3', Encoder(4)), #SFF
    ('encoder4', Encoder(5)), #CHOW_LAST_12MOS
    ('encoder5', Encoder(6)), #SPRINKLER_STATUS
    ('scaler', scaler),
    # add your estimator here
    ('clf', AdaBoostClassifier(n_estimators=50, random_state=42, base_estimator=DecisionTreeClassifier(random_state=42, max_depth=1)))
])

In [50]:
# business_model = Pipeline([
#     ('features', business_features),
#     # add your estimator here
#     ('ploy', PolynomialFeatures(2)),
#     ('lr', LogisticRegression())
# ])

In [67]:
business_model.fit(data, fine_counts > 0)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('simple',
                                                 Pipeline(steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                                                   'RESTOT',
                                                                                                   'INHOSP',
                                                                                                   'CCRC_FACIL',
                                                                                                   'SFF',
                                                                                                   'CHOW_LAST_12MOS',
                                                                                                   'SPRINKLER_STATUS',
                                                       

In [68]:
grader.score.ml__business_model(positive_probability(business_model))

Your score: 1.025


## Question 5: survey_results

Surveys reveal safety and health deficiencies at nursing homes that may indicate risk for incidents (and penalties). CMS routinely makes surveys of nursing homes. Build a model that combines the `business_features` of each facility with its cycle 1 survey results, as well as the time between the cycle 1 and cycle 2 survey to predict the cycle 2 total score.

First, let's create a transformer to calculate the difference in time between the cycle 1 and cycle 2 surveys.

In [38]:
# class TimedeltaTransformer(BaseEstimator, TransformerMixin):
#     def __init__(self, t1_col, t2_col):
#         self.t1_col = t1_col
#         self.t2_col = t2_col

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         if not isinstance(X, pd.DataFrame):
#             X = pd.DataFrame(X)
#         Xr = X.copy() # copy() to prevent SettingWithCopyWarning
#         Xr['date1'] = pd.to_datetime(Xr[self.t1_col]) 
#         Xr['date2'] = pd.to_datetime(Xr[self.t2_col])
        
#         return Xr[['date1','date2']].apply(lambda x: (x[0] - x[1]).days, axis=1).to_frame(name='TIMEDELTA')
class TimedeltaTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, t1_col, t2_col):
        self.t1_col = t1_col
        self.t2_col = t2_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance (X, pd.DataFrame):
            X = pd.DataFrame(X)
        results = (pd.to_datetime(X[self.t1_col]) - pd.to_datetime(X[self.t1_col]))
        
        results = results.apply(lambda X:X.days).values.reshape(-1, 1)
        
        return results

In [39]:
cycle_1_date = 'CYCLE_1_SURVEY_DATE'
cycle_2_date = 'CYCLE_2_SURVEY_DATE'
time_feature = TimedeltaTransformer(cycle_1_date, cycle_2_date)

In the cell below we'll collect the cycle 1 survey features.

In [40]:
cycle_1_cols = ['CYCLE_1_DEFS', 'CYCLE_1_NFROMDEFS', 'CYCLE_1_NFROMCOMP',
                'CYCLE_1_DEFS_SCORE', 'CYCLE_1_NUMREVIS',
                'CYCLE_1_REVISIT_SCORE', 'CYCLE_1_TOTAL_SCORE']
cycle_1_features = ColumnSelectTransformer(cycle_1_cols)

In [45]:

business_features = FeatureUnion([
    ('simple', simple_features),
    ('categorical', categorical_features)
])

In [46]:
# from sklearn.preprocessing import PolynomialFeatures

# business_feature = FeatureUnion([

#     ('simple', simple_features),
#     ('categorical', categorical_features)
    
# ])

# business = "business_features"

from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(Lasso(max_iter = 1000), param_grid = {'alpha': np.arange(0, 3.5, 0.5)}, cv=5, n_jobs=4, verbose=1)



survey_model = Pipeline([
    ('features', FeatureUnion([
        ('business', business_features),
        ('survey', cycle_1_features),
        ('time', time_feature)
    ])),
    # add your estimator here
    ('poly', PolynomialFeatures(2)),
    ('decomp', TruncatedSVD(40)),
    ('gs', gs)
])

In [47]:
survey_model.fit(data, cycle_2_score.astype(int))

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  35 out of  35 | elapsed:   10.4s finished


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('business',
                                                 FeatureUnion(transformer_list=[('simple',
                                                                                 Pipeline(steps=[('cst',
                                                                                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                                                                                   'RESTOT',
                                                                                                                                   'INHOSP',
                                                                                                                                   'CCRC_FACIL',
                                                                                                                                   'SFF',
       

In [48]:
grader.score.ml__survey_model(survey_model.predict)

Your score: 1.027


*Copyright &copy; 2021 WorldQuant University. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*